# Tensorflow MNIST訓練

In [1]:
import numpy as np
import tensorflow as tf

In [5]:
# MNISTデータロード
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [6]:
# 正規化
x_train = x_train.reshape((60000, 28, 28, 1))
x_test = x_test.reshape((10000, 28, 28, 1))
x_train = x_train / 255.0
x_test = x_test / 255.0

In [7]:
# 多層パーセプトロン(MLP)構築
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((28, 28, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# モデル可視化
model.summary()

In [9]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# 訓練
EPOCHS=50
model.fit(
    x_train,
    y_train,
    epochs=EPOCHS,
    batch_size=128,
    validation_data=(x_test, y_test)
)

In [ ]:
# モデル評価
val_loss, val_acc = model.evaluate(x_test, y_test, batch_size=128)

In [ ]:
# モデル保存
model.save('saved_model', save_format='tf')

# Tensorflow MNIST推論テスト

In [13]:
# 保存したモデルのロード
load_model = tf.keras.models.load_model('saved_model')

In [ ]:
# テスト画像ダウンロード
!wget 'https://raw.githubusercontent.com/Kazuhito00/tensorflow2-keras-learn/master/01_mlp_mnist/image/03.jpg'

In [15]:
# テスト画像を1枚ロード
from IPython.display import Image, display_png
from PIL import Image

img = Image.open('03.jpg')
img = img.resize((28, 28))
display_png(img)

In [33]:
# 入力画像成形、および正規化
x = img.convert('L')
x = np.asarray(x)
x = x.reshape(-1, 28, 28, 1)
x = x.astype('float32')
x /= 255

In [ ]:
print(x)

In [ ]:
%%time
# 推論実行
predict_result = load_model.predict(x)
# 推論結果表示
print(predict_result)
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

# ONNXへ変換

In [ ]:
!pip install onnxruntime
!pip install tf2onnx

In [ ]:
!python -m tf2onnx.convert --saved-model='saved_model' \
                           --opset=9 \
                           --output='mnist.onnx'

In [40]:
import onnxruntime
sess_onnxruntime = onnxruntime.InferenceSession('mnist.onnx')

In [ ]:
res = sess_onnxruntime.run(output_names=['Identity:0'], input_feed={'input_1:0': x})
print("The digit is classified as \"%s\" in ONNXRruntime"%np.argmax(res))